In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import outline_site, make_symmetric_sites, combine_sites, plot_density, Site, plot_bulk_counts
plt.rcParams['axes.grid'] = False 

In [ ]:
# Enrichment plot colormap
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

# Read and plot the distribution
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 50
root = Path(f"../accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

In [ ]:
probabilities[0]

In [ ]:

fig, axes = plt.subplots(2, 1, figsize=(3,6), sharex=True, sharey=True)

area = 50
#root = Path(f"/Users/ezry/Projects/Polar_Binning_DeltaG/accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_cheng = frequencies/np.sum(frequencies)
axes[0] = plot_bulk_counts(axes[0], bulk_counts, area, lw=4)
axes[0].set_xlabel(None)
axes[0].set_ylabel("P(n) Site 1")

area = 96
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_ulens = frequencies/np.sum(frequencies)
axes[1] = plot_bulk_counts(axes[1], bulk_counts, area, lw=4)
axes[1].set_xlabel("n")
axes[1].set_ylabel("P(n) Site 2")

fig.subplots_adjust(left=0.2)
plt.savefig(f"POPG_bulk_distribution_combo.pdf")

In [ ]:
np.sum(probabilities_ulens[12:])

In [ ]:
np.sum(probabilities_cheng[12:])

# Get Site Distributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import make_simple_site, combine_sites, plot_density, make_symmetric_sites

In [ ]:
root = Path("/Users/ezry/Projects/ELIC_PE_PG/aggregated")
replicas = ["liam", "rep1", "rep3"]
lipid = "POPG"

In [ ]:
leaf = "upp"
MB =11
AB = 0
EB = 2
fig, axes = plt.subplots(3,5, figsize=(15,10), sharex=True, sharey=True)
axis = 0
p_occ = pd.DataFrame(columns=["AB", "MB", "EB"], index=np.arange(0,14))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    try:
        cheng_sites = make_symmetric_sites(the_data, 4, 3, 24, 30, Ntheta, dr, dth, exrho, frames, sitename="Cheng Site ")
    except:
        print(f"Error on rep {rep}")
    #cheng_site_symmetric = combine_sites(cheng_sites, exrho, "Cheng Sites, Symmetric", symmetric=True)
    #cheng_sites.append(cheng_site_symmetric)
    for site in cheng_sites:
        site.Npeak = 0
        site.expPunocc = 0.302

    for site, ax, idx in zip(cheng_sites, axes.flatten()[axis:], np.arange(0,5)):
        p_occ.at[idx+axis, "AB"] = np.sum(site.densities[AB+1:])
        p_occ.at[idx+axis, "MB"] = np.sum(site.densities[MB+1:])
        p_occ.at[idx+axis, "EB"] = np.sum(site.densities[EB+1:])
        ax = plot_density(site, ax)

    axis+=len(cheng_sites)
 
fig.tight_layout()
plt.savefig(data_root.joinpath(f"Cheng_sites.pdf"))

In [ ]:
np.average(p_occ, axis=0)

In [ ]:
np.std(p_occ, axis=0)/np.sqrt(15)

In [ ]:
leaf = "upp"
lipid="POPG"
fig, ax = plt.subplots(1,1, figsize=(5,5))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    cheng_sites = make_symmetric_sites(the_data, 4, 3, 24, 30, Ntheta, dr, dth, exrho, frames, sitename="Cheng Site ")
    #cheng_site_symmetric = combine_sites(cheng_sites, exrho, "Cheng Sites, Symmetric", symmetric=True)
    #cheng_sites.append(cheng_site_symmetric)
    for site in cheng_sites:
        site.Npeak = 0
        site.expPunocc = 0.302

    dummy_site = cheng_sites[0]
    dummy_site.title=f"average over sites, {lipid}, {leaf}er leaflet, Cheng site"
    temp = [site.densities for site in cheng_sites]
    densities = [np.pad(dens, (0,18-len(dens))) for dens in temp]
    dummy_site.densities = np.average(densities, axis=0)

    ax = plot_density(dummy_site, ax)

plt.savefig(root.joinpath(f"Cheng_sites_averaged_{lipid}.pdf"))

In [ ]:
leaf = "low"
MB =11
AB = 0
EB = 0
fig, axes = plt.subplots(3,5, figsize=(15,10), sharex=True, sharey=True)
axis = 0
p_occ = pd.DataFrame(columns=["AB", "MB", "EB"], index=np.arange(0,14))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    try:
        ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
    except:
        print(f"Error on rep {rep}")
    #ulens_sites.append(ulens_site_symmetric)
    for site in ulens_sites:
        site.Npeak=0
        site.expPunocc = 0.187

    for site, ax, idx in zip(ulens_sites, axes.flatten()[axis:], np.arange(0,5)):
        p_occ.at[idx+axis, "AB"] = np.sum(site.densities[AB+1:])
        p_occ.at[idx+axis, "MB"] = np.sum(site.densities[MB+1:])
        p_occ.at[idx+axis, "EB"] = np.sum(site.densities[EB+1:])
        ax = plot_density(site, ax)

    axis+=len(cheng_sites)
 
fig.tight_layout()
plt.savefig(data_root.joinpath(f"Ulens_sites.pdf"))

In [ ]:
np.average(p_occ, axis=0)

In [ ]:
np.std(p_occ, axis=0)/np.sqrt(15)

In [ ]:
# leaf = "low"
# lipid = "POPG"
# for rep in replicas:
#     data_root = root.joinpath(rep)
#     fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

#     num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
#     the_data = np.loadtxt(fpath, skiprows=1)
#     rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

#     ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
#     #ulens_site_symmetric = combine_sites(ulens_sites, exrho, "Ulens Sites, Symmetric", symmetric=True)
#     #ulens_sites.append(ulens_site_symmetric)
#     for site in ulens_sites:
#         site.Npeak=0
#         site.expPunocc = 0.187

#     fig, axes = plt.subplots(3,2, figsize=(15,15))
#     for site, ax in zip(ulens_sites, axes.flatten()):
#         ax = plot_density(site, ax)

#     plt.savefig(data_root.joinpath(f"Ulens_sites.pdf"))

In [ ]:
# leaf = "low"
# for rep in replicas:
#     data_root = root.joinpath(rep)
#     fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

#     num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
#     the_data = np.loadtxt(fpath, skiprows=1)
#     rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

#     ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
#     ulens_sites.append(ulens_site_symmetric)
#     for site in ulens_sites:
#         site.Npeak=0
#         site.expPunocc = 0.187

#     dummy_site = ulens_sites[0]
#     dummy_site.title="average over sites"
#     temp = [site.densities for site in ulens_sites]
#     densities = [np.pad(dens, (0,50-len(dens))) for dens in temp]
#     dummy_site.densities = np.average(densities, axis=0)

#     fig, ax = plt.subplots(1,1, figsize=(10,10))
#     ax = plot_density(dummy_site, ax)

#     plt.savefig(data_root.joinpath(f"Ulens_sites_averaged.pdf"))

In [ ]:
leaf = "low"
lipid = "POPG"
fig, ax = plt.subplots(1,1, figsize=(5,5))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
    #ulens_sites.append(ulens_site_symmetric)
    for site in ulens_sites:
        site.Npeak=0
        site.expPunocc = 0.187

    dummy_site = ulens_sites[0]
    dummy_site.title="average over sites"
    temp = [site.densities for site in ulens_sites]
    densities = [np.pad(dens, (0,30-len(dens))) for dens in temp]
    dummy_site.densities = np.average(densities, axis=0)

    ax = plot_density(dummy_site, ax)

plt.savefig(root.joinpath(f"Ulens_sites_averaged_{lipid}.pdf"))